In [8]:
import numpy as np
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.


In [4]:
ROOT_DIR = "train_data"

In [72]:
fakes = os.listdir("train_data/celebdf/face_crops/id0_id1_0009")
len(fakes)

52

In [66]:
data = []
print(len(data))

0


In [73]:

for f in fakes:
    data.append({
        "video" : "id0_id1_0009",
        "file" : f, 
        "label" : 1, 
        "original" : "id0_0009",
        "frame" : f.split('_')[0],
        "fold" : -1,
        "dataset" : "celebdf",
    })
print(len(data))

187


In [74]:
df = pd.DataFrame(data)
df

,video,file,label,original,frame,fold,dataset
0,00000,0_0.png,0,00000,0,-1,celebdf
1,00000,100_0.png,0,00000,100,-1,celebdf
2,00000,10_0.png,0,00000,10,-1,celebdf
3,00000,110_0.png,0,00000,110,-1,celebdf
4,00000,120_0.png,0,00000,120,-1,celebdf
...,...,...,...,...,...,...,...
182,id0_id1_0009,510_0.png,1,id0_0009,510,-1,celebdf
183,id0_id1_0009,60_0.png,1,id0_0009,60,-1,celebdf
184,id0_id1_0009,70_0.png,1,id0_0009,70,-1,celebdf
185,id0_id1_0009,80_0.png,1,id0_0009,80,-1,celebdf


In [75]:
from sklearn import model_selection

df["fold"] = -1    
df = df.sample(frac=1).reset_index(drop=True)
y = df.label.values
kf = model_selection.StratifiedKFold(n_splits=5)

for f, (t_, v_) in enumerate(kf.split(X=df, y=y)):
    df.loc[v_, 'fold'] = f

df.to_csv("celebdf.csv", index=False)

CHECK

In [76]:
ddf = pd.read_csv('celebdf.csv')
print(len(ddf))

for i, row in ddf.iterrows():
    video, file, label, original, frame, fold, dataset = row
    path = os.path.join(ROOT_DIR, dataset, "face_crops", video, file)
    # if(video == "deepfake_003_000"):
    #     print(path)
    if os.path.exists(path) == False:
        print(path)
        ddf.drop(i, inplace = True)
    # break
print(len(ddf))

187
187


In [20]:
ddf.to_csv("ffpp.csv", index=False)

In [11]:
from skimage.metrics import structural_similarity as compare_ssim

dataframe = pd.read_csv('ffpp.csv')
for i, row in dataframe.iterrows():
    video, file, label, original, frame, fold, dataset = row
    if label == 1:
        fake_path = os.path.join(ROOT_DIR, dataset, "face_crops", video, file)
        real_path = os.path.join("I:/DEEPFAKE/FF++/ff_data/face_crops", original, file)

        fake = cv2.imread(fake_path)
        fake = cv2.cvtColor(fake, cv2.COLOR_BGR2RGB)

        real = cv2.imread(real_path)
        real = cv2.cvtColor(real, cv2.COLOR_BGR2RGB)
        
        d, a = compare_ssim(real, fake, multichannel=True, full=True)
        a = 1 - a
        diff = (a * 255).astype(np.uint8)
        diff = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)

        diff_path = f"train_data/ffpp/diffs/{video}"
        os.makedirs(diff_path, exist_ok=True)

        cv2.imwrite(os.path.join(diff_path,file[:-4]+"_diff.png"), diff)

        